In [1]:
# ipython settings
%load_ext autoreload
%autoreload 2
import warnings

warnings.filterwarnings('ignore')

# 3. Synthetic experiments with Symbolic Pursuit

In this notebook, we shall reproduce one of the experiments from Section 6.1 of the paper.
The idea is to start with a linear pseudo black-box for which the importance vector is known unambiguously and see which interpretability methods identifies this vector the most precisely. Let us start by the useful imports.


In [2]:
from symbolic_pursuit.models import SymbolicRegressor  # our symbolic model class
from sklearn.metrics import mean_squared_error # we are going to assess the quality of the model based on the generalization MSE
from sympy import init_printing # We use sympy to display mathematical expresssions 
import numpy as np # we use numpy to deal with arrays
import lime 
import lime.lime_tabular
init_printing()

We now define a linear pseudo black-box $f$ defined on a 3 dimensional feature space.

$$ f(x_1,x_2,x_3)= x_1 + 2 \cdot x_2 + 3 \cdot x_3$$ 

The importance vector associated to this model is trivially given by $\beta = (1,2,3)$ In this case, we shall keep it unnormalised, unlike in the main paper as we deal with few examples. Let us translate this in Python. 

In [3]:
def f(X):
    return X[:, 0]+2*X[:,1]+3*X[:,2]

dim_X = 3

Now draw uniformly 100 test points  that we will feed to a *LIME* explainer <cite data-cite="2480681/WCEBQ7N9"></cite> and to train a Symbolic model.

In [4]:
n_pts = 100
X = np.random.uniform(0, 1, (n_pts, dim_X))

Now we draw 10 test ponits $x_{test} \equiv U([0,1]^3)$ that we are going to use in order to evaluate the perfomances of both explainers on unseen data.

In [5]:
n_test = 10
X_test = np.random.uniform(0, 1, (n_test, dim_X))

Since LIME produces importance vectors with entries in the form $(feature \ domain , importance)$ for each feature appearing in decreasing order of importance, we implement a function which identifies the feature from the first entry of the tuple and who sorts the importances in the form $(importance(x_1), importance(x_2), importance(x_3))$.

In [6]:
def order_weights(exp_list):
    ordered_weights = [0 for _ in range(dim_X)]
    for tup in exp_list:
        feature_id = int(tup[0].split('x_')[1][0])
        ordered_weights[feature_id-1] = tup[1]    
    return ordered_weights    

We are now ready to extract the feature importance for our 10 test points as predicted by the LIME explainer :

In [7]:
lime_weight_list = []
explainer = lime.lime_tabular.LimeTabularExplainer(X, 
                                                   feature_names=["x_"+str(k) for k in range(1,dim_X+1)], 
                                                   class_names=['f'], 
                                                   verbose=True,
                                                   mode='regression')

for i in range(n_test):
    exp = explainer.explain_instance(X_test[i], f, num_features=dim_X)
    lime_weight_list.append(order_weights(exp.as_list()))  
                            
print(lime_weight_list)    

Intercept 2.9633864824154013
Prediction_local [3.01912861]
Right: 3.2696169075607773
Intercept 2.980957806196677
Prediction_local [3.00064701]
Right: 2.9004087168145474
Intercept 2.983420551028581
Prediction_local [2.93887354]
Right: 3.032114925664124
Intercept 3.227093690455886
Prediction_local [2.20298927]
Right: 2.02904679760283
Intercept 2.3159003893474313
Prediction_local [4.91384487]
Right: 4.567985336594555
Intercept 2.3748488012536297
Prediction_local [4.7186416]
Right: 4.332309728908863
Intercept 3.0287982351952523
Prediction_local [2.73081149]
Right: 2.486540644428578
Intercept 3.4568743142026808
Prediction_local [1.49995776]
Right: 1.6580632721932194
Intercept 3.310784896190186
Prediction_local [1.9837895]
Right: 2.3155798648957977
Intercept 2.8887678816610647
Prediction_local [3.21240778]
Right: 3.4458320426771762
[[0.4988391438716656, -0.9786651392748903, 0.5355681243471734], [-0.4700395362925193, -0.979077727873307, 1.4688064648534784], [-0.4621630629088637, 0.95431667277

As we can see from the last output, which is the list of predicted importance vectors, LIME seems to produce a big variety of importance vectors. This is suprising for a global linear model. We also note that the relative importance seem inconsistent with the true importance vector $\beta$ defined above. Let us now train a Symbolic model for $f$ based on our training set.

In [8]:
symbolic_model = SymbolicRegressor(maxiter=3,
                 eps=1.0e-4)
symbolic_model.fit(f, X)

Model created with the following hyperparameters :
 loss_tol=0.001 
 ratio_tol=0.9 
 maxiter=3 
 eps=0.0001 
 random_seed=42
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
Now working on term number  1 .
Now working on hyperparameter tree number  1 .
         Current function value: 2.504912
         Iterations: 3
         Function evaluations: 48
         Gradient evaluations: 6
Now working on hyperparameter tree number  2 .
         Current function value: 1.336307
         Iterations: 3
         Function evaluations: 70
         Gradient evaluations: 7
Now working on hyperparameter tree number  3 .
         Current function value: 1.020032
         Iterations: 3
         Function evaluations: 209
         Gradient evaluations: 19
The tree number  3  was selected as the best.
Backfitting complete.
The current model has the following expression:  1.17010410164818*meijerg(((0.831735241075464,), (0.105422258418036,)), ((0.15154198089

We now ask our symbolic model to predict the importance vectors for each test point.

In [9]:
symbolic_weight_list = [] 
for k in range(n_test):
    symbolic_weight_list.append(symbolic_model.get_feature_importance(X_test[k]))
    

In [10]:
print(symbolic_weight_list)

[[0.304103643986965, 0.985616070373492, 1.85934322817885], [0.409219703948425, 0.858536332262576, 1.84096578696438], [3.60238638537070, -0.356466444262796, 5.30660464576506], [0.797220991428648, 1.05077619686094, 2.75469021861172], [0.428319537688384, 0.742949895127590, 1.69920150617097], [0.473655382235505, 0.735160133584799, 1.76311545037284], [0.659985283016554, 0.842700811470965, 2.22613245324729], [0.752718390210156, 1.11440794418485, 2.76811045260715], [0.967332591725389, 0.798453090524580, 2.65791010115301], [0.611074149150842, 0.752754346002077, 2.01661845208323]]


As we can see, our results appear to be always consistent and very close to the true importance vector $\beta$.

## References<div class="cite2c-biblio"></div>

<div class="cite2c-biblio"></div>